In [1]:
%pwd

'C:\\NN-P4\\python-notebooks'

###Imports and Read CSV

In [2]:
import time
start = time.time()
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from keras.callbacks import History # to get train accuracy
history = History()

In [3]:
#nPrintML creates a copy of dataset in pkl picles format. Convert to CSV 1st.
# Read the PKL
# df_train = pd.read_pickle('../datasets/nprint-raw/os-detection/X.pkl')
# df_test = pd.read_pickle('../datasets/nprint-raw/os-detection/X_val.pkl')
# label_train = pd.read_pickle('../datasets/nprint-raw/os-detection/y.pkl')
# label_test = pd.read_pickle('../datasets/nprint-raw/os-detection/y_val.pkl')
# # Save a copy as CSV
# df_train.to_csv('../datasets/nprint-raw/os-detection/X.csv')
# df_test.to_csv('../datasets/nprint-raw/os-detection/X_val.csv')
# label_train.to_csv('../datasets/nprint-raw/os-detection/y.csv')
# label_test.to_csv('../datasets/nprint-raw/os-detection/y_val.csv')
# Read CSV
df_train = pd.read_csv('../datasets/nprint-raw/os-detection/X.csv')
df_test = pd.read_csv('../datasets/nprint-raw/os-detection/X_val.csv')
label_train = pd.read_csv('../datasets/nprint-raw/os-detection/y.csv')
label_test = pd.read_csv('../datasets/nprint-raw/os-detection/y_val.csv')
#Drop colum with pcap sequence number
df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
#Rename Unnamed Collumn
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)
#Read file with ordered list of attributes ranked by the autoML feature importance algorithm
features_rankings_df = pd.read_csv('../datasets/nprint-raw/os-detection/feature-importance.csv')
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

In [4]:
display(df_train)
display(df_test)
display(label_train)
display(label_test)
print(label_train["label"].value_counts())
print(label_test["label"].value_counts())
print(f"df_train shape: {df_train.shape} \n columns: {df_train.columns.values}")
print(f"df_test shape: {df_test.shape} \n columns: {df_test.columns.values}")
display(features_rankings_df.head(40))

,pkt_0_ipv4_tos_1,pkt_0_ipv4_tos_2,pkt_0_ipv4_tos_4,pkt_0_ipv4_tl_3,pkt_0_ipv4_tl_4,pkt_0_ipv4_tl_5,pkt_0_ipv4_tl_6,pkt_0_ipv4_tl_7,pkt_0_ipv4_tl_8,pkt_0_ipv4_tl_9,...,pkt_9_tcp_opt_310,pkt_9_tcp_opt_311,pkt_9_tcp_opt_312,pkt_9_tcp_opt_313,pkt_9_tcp_opt_314,pkt_9_tcp_opt_315,pkt_9_tcp_opt_316,pkt_9_tcp_opt_317,pkt_9_tcp_opt_318,pkt_9_tcp_opt_319
0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,0,0,0,0,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,0,0,0,0,0,0,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,0,1,0,0,0,0,0,0,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6685,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6686,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6687,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6688,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,pkt_0_ipv4_tos_1,pkt_0_ipv4_tos_2,pkt_0_ipv4_tos_4,pkt_0_ipv4_tl_3,pkt_0_ipv4_tl_4,pkt_0_ipv4_tl_5,pkt_0_ipv4_tl_6,pkt_0_ipv4_tl_7,pkt_0_ipv4_tl_8,pkt_0_ipv4_tl_9,...,pkt_9_tcp_opt_310,pkt_9_tcp_opt_311,pkt_9_tcp_opt_312,pkt_9_tcp_opt_313,pkt_9_tcp_opt_314,pkt_9_tcp_opt_315,pkt_9_tcp_opt_316,pkt_9_tcp_opt_317,pkt_9_tcp_opt_318,pkt_9_tcp_opt_319
0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,0,0,0,0,0,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
740,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
741,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
742,0,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,label
0,2
1,2
2,2
3,1
4,0
...,...
6685,2
6686,1
6687,2
6688,1


,label
0,1
1,2
2,1
3,1
4,2
...,...
739,1
740,2
741,1
742,2


label
1    3522
2    2631
0     537
Name: count, dtype: int64
label
1    391
2    293
0     60
Name: count, dtype: int64
df_train shape: (6690, 5121) 
 columns: ['pkt_0_ipv4_tos_1' 'pkt_0_ipv4_tos_2' 'pkt_0_ipv4_tos_4' ...
 'pkt_9_tcp_opt_317' 'pkt_9_tcp_opt_318' 'pkt_9_tcp_opt_319']
df_test shape: (744, 5121) 
 columns: ['pkt_0_ipv4_tos_1' 'pkt_0_ipv4_tos_2' 'pkt_0_ipv4_tos_4' ...
 'pkt_9_tcp_opt_317' 'pkt_9_tcp_opt_318' 'pkt_9_tcp_opt_319']


,Unnamed: 0,importance,stddev,p_value,n,p99_high,p99_low
0,pkt_0_ipv4_ttl_0,0.480914,0.017648,9.668152e-15,10,0.499051,0.462777
1,pkt_2_tcp_sprt_1,0.023118,0.006393,5.796828e-07,10,0.029688,0.016548
2,pkt_8_tcp_sprt_1,0.020968,0.005155,2.127980e-07,10,0.026266,0.015670
3,pkt_9_tcp_sprt_1,0.020161,0.004057,3.762506e-08,10,0.024331,0.015992
4,pkt_1_tcp_opt_44,0.013844,0.003804,5.496057e-07,10,0.017754,0.009934
5,pkt_9_tcp_wsize_0,0.013441,0.001267,4.586664e-11,10,0.014743,0.012139
6,pkt_4_tcp_sprt_2,0.012903,0.002470,2.435383e-08,10,0.015442,0.010365
7,pkt_8_tcp_sprt_2,0.012903,0.003304,3.016954e-07,10,0.016299,0.009507
8,pkt_0_tcp_opt_36,0.004570,0.000940,4.545592e-08,10,0.005536,0.003604
9,pkt_0_ipv4_ttl_1,0.004167,0.002048,6.026156e-05,10,0.006272,0.002062


In [5]:
#make a list with only the best features

feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:32]
print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]

display(df_train)
display(df_test)

['pkt_0_ipv4_ttl_0', 'pkt_2_tcp_sprt_1', 'pkt_8_tcp_sprt_1', 'pkt_9_tcp_sprt_1', 'pkt_1_tcp_opt_44', 'pkt_9_tcp_wsize_0', 'pkt_4_tcp_sprt_2', 'pkt_8_tcp_sprt_2', 'pkt_0_tcp_opt_36', 'pkt_0_ipv4_ttl_1', 'pkt_6_tcp_sprt_1', 'pkt_0_tcp_opt_40', 'pkt_0_tcp_opt_42', 'pkt_1_tcp_wsize_1', 'pkt_5_tcp_opt_40', 'pkt_2_tcp_wsize_0', 'pkt_1_tcp_sprt_1', 'pkt_9_tcp_rst_0', 'pkt_7_tcp_opt_34', 'pkt_2_tcp_opt_36', 'pkt_2_tcp_opt_40', 'pkt_3_tcp_opt_34', 'pkt_3_tcp_opt_40', 'pkt_2_tcp_opt_34', 'pkt_4_tcp_opt_36', 'pkt_5_tcp_wsize_0', 'pkt_6_tcp_sprt_2', 'pkt_0_tcp_sprt_1', 'pkt_9_tcp_wsize_14', 'pkt_1_tcp_sprt_2', 'pkt_1_tcp_opt_46', 'pkt_1_ipv4_cksum_6']


,pkt_0_ipv4_ttl_0,pkt_2_tcp_sprt_1,pkt_8_tcp_sprt_1,pkt_9_tcp_sprt_1,pkt_1_tcp_opt_44,pkt_9_tcp_wsize_0,pkt_4_tcp_sprt_2,pkt_8_tcp_sprt_2,pkt_0_tcp_opt_36,pkt_0_ipv4_ttl_1,...,pkt_3_tcp_opt_40,pkt_2_tcp_opt_34,pkt_4_tcp_opt_36,pkt_5_tcp_wsize_0,pkt_6_tcp_sprt_2,pkt_0_tcp_sprt_1,pkt_9_tcp_wsize_14,pkt_1_tcp_sprt_2,pkt_1_tcp_opt_46,pkt_1_ipv4_cksum_6
0,1,1,1,1,-1,0,0,0,-1,0,...,-1,-1,-1,1,0,1,0,0,-1,0
1,1,1,1,1,-1,0,0,0,-1,0,...,-1,-1,-1,0,0,1,1,0,-1,1
2,1,1,1,1,-1,1,0,0,-1,0,...,-1,0,-1,0,0,1,0,0,-1,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,1
4,0,1,1,1,0,0,0,0,1,1,...,1,1,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6685,1,1,1,1,-1,0,0,0,-1,0,...,-1,-1,-1,0,0,1,1,0,-1,0
6686,0,1,1,1,-1,0,1,0,-1,1,...,-1,-1,-1,0,0,1,0,0,-1,1
6687,1,1,1,1,0,0,0,0,-1,0,...,-1,0,-1,0,0,1,0,0,1,0
6688,0,1,1,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0


,pkt_0_ipv4_ttl_0,pkt_2_tcp_sprt_1,pkt_8_tcp_sprt_1,pkt_9_tcp_sprt_1,pkt_1_tcp_opt_44,pkt_9_tcp_wsize_0,pkt_4_tcp_sprt_2,pkt_8_tcp_sprt_2,pkt_0_tcp_opt_36,pkt_0_ipv4_ttl_1,...,pkt_3_tcp_opt_40,pkt_2_tcp_opt_34,pkt_4_tcp_opt_36,pkt_5_tcp_wsize_0,pkt_6_tcp_sprt_2,pkt_0_tcp_sprt_1,pkt_9_tcp_wsize_14,pkt_1_tcp_sprt_2,pkt_1_tcp_opt_46,pkt_1_ipv4_cksum_6
0,0,0,0,0,0,0,0,0,0,1,...,-1,-1,-1,0,0,0,0,0,0,0
1,1,1,1,1,-1,0,0,0,-1,0,...,-1,-1,-1,0,0,1,0,0,-1,0
2,0,0,0,0,0,0,1,1,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,1,1,1,1,-1,1,1,1,0,0,...,-1,-1,-1,0,1,1,1,1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0,0,1,0,0,0,1,1,-1,1,...,0,0,0,0,1,0,1,0,1,0
740,1,1,1,1,-1,0,0,0,-1,0,...,0,-1,1,0,0,1,0,0,-1,1
741,0,0,1,0,0,0,1,1,0,1,...,0,0,0,0,1,0,0,1,1,0
742,1,1,1,1,-1,0,0,0,-1,0,...,-1,-1,-1,0,0,1,0,0,-1,0


In [6]:
# Model Parameters
number_of_attributes = df_train.shape[1]
hidden_layer_nodes = 32
output_layer_nodes = 3

# Traing Parameters
EPOCHS = 500
BATCH_SIZE = 500

# Run the NN Training multiple times to get a nice mean score of accuracy results
NUMBER_OF_NN_REPETITIONS = 10

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [7]:
# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Set the 
normalizer = tf.keras.layers.Normalization(axis=-1)
# Normalize the data
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

In [8]:
# Execute Trainning and Testing
#model = get_basic_model()
train_accuracy = []
test_accuracy = []

for i in range(NUMBER_OF_NN_REPETITIONS):
  print(f"NN REPETITION {i+1}")
  model = get_basic_model()
  history = model.fit(df_train, label_train, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
  one_train_accuracy = history.history['accuracy']

  test_loss, one_test_accuracy =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
  train_accuracy.append(one_train_accuracy[-1])
  test_accuracy.append(one_test_accuracy)
  print(f"{i+1}: train_accuracy:{train_accuracy} test_accuracy:{test_accuracy}")
  tf_predictions_probabilities = model.predict(df_test)
  #tf_loss, tf_acc =  model.evaluate(numeric_features_test,  label_test, verbose=2, batch_size=BATCH_SIZE)

  #Confusion Matrix
  tf_predictions = []
  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)
    j_max = x.argmax()
    tf_predictions.append(j_max)
  conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
  print(conf_m)
    
  # Precision, Recall, F1 Score Computation
  label_test = np.array(label_test)  # True labels
  tf_predictions = np.array(tf_predictions)  # Predicted labels
  # Number of classes
  num_classes = output_layer_nodes  
  # Initialize metrics
  precision_list = []
  recall_list = []
  f1_list = []
  auc_list = []    
  # Calculate metrics for each class
  for i in range(num_classes):
      # Binary labels for class i (one-vs-rest)
      true_binary = (label_test == i)  # This is a boolean series now
      pred_binary = (tf_predictions == i)
      
      # True Positives, False Positives, False Negatives
      TP = np.sum(true_binary & pred_binary)
      FP = np.sum(~true_binary & pred_binary)
      FN = np.sum(true_binary & ~pred_binary)
      
      # Precision, Recall, F1 for class i
      precision = TP / (TP + FP) if (TP + FP) > 0 else 0
      recall = TP / (TP + FN) if (TP + FN) > 0 else 0
      f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
      
      # Store the metrics
      precision_list.append(precision)
      recall_list.append(recall)
      f1_list.append(f1)
            
  print(f'precision_list: {precision_list}, recall_list: {recall_list}, f1_list: {f1_list}')
  # Calculate macro-averaged metrics
  precision_macro = np.sum(precision_list) #ORIGINAL FORMULA WAS MEAN
  recall_macro = np.sum(recall_list)
  f1_macro = np.sum(f1_list)
  print(f'precision_macro: {precision_macro}, recall_macro: {recall_macro}, f1_macro: {f1_macro}')

#After n repetitions of training and testing, calculate statistics
train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
train_max = max(train_accuracy)
train_min = min(train_accuracy)
train_std = np.std(train_accuracy)
test_accuracy_mean = sum(test_accuracy) / len(test_accuracy)
test_max = max(test_accuracy)
test_min = min(test_accuracy)
test_std = np.std(test_accuracy)

print(f'accuracies_train: {train_accuracy}, accuracies_test: {test_accuracy}')
print(f'train_accuracy_mean: {train_accuracy_mean*100:.2f}, train_max: {train_max*100:.2f}, train_min: {train_min*100:.2f}, train_std: {train_std*100:.2f} \n test_accuracy_mean: {test_accuracy_mean*100:.2f}, test_max: {test_max*100:.2f}, test_min: {test_min*100:.2f}, test_std: {test_std*100:.2f}')



NN REPETITION 1
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9984 - loss: 0.0146 
1: train_accuracy:[0.9998505115509033] test_accuracy:[0.9986559152603149]
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 60   0   0]
 [  1 390   0]
 [  0   0 293]], shape=(3, 3), dtype=int32)
precision_list: [0.08064516129032258, 0.5255376344086021, 0.39381720430107525], recall_list: [0.08198924731182795, 0.5241935483870968, 0.39381720430107525], f1_list: [0.08131165022660623, 0.5248647309074388, 0.39381720430107525]
precision_macro: 1.0, recall_macro: 1.0, f1_macro: 0.9999935854351203
NN REPETITION 2
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9984 - loss: 0.0143 
2: train_accuracy:[0.9998505115509033, 0.9998505115509033] test_accuracy:[0.9986559152603149, 0.9986559152603149]
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 60   0   0]
 [  1 390   0]
 [  0   0 293]], shape=(3, 3), dtype=int32)
precision_list: [0.08064516129032258, 0.5255376344086021, 0.39381720430107525], recall_

In [9]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "training_dataset_shape":  str(df_train.shape),
    "test_dataset_shape": str(df_test.shape),
    "number_of_attributes": number_of_attributes,
    "n_epoch":EPOCHS,
    "accuracy_train": train_accuracy_mean,
    "train_max": train_max,
    "train_min": train_min,
    "train_std": train_std,
    "accuracy_test": test_accuracy_mean,
    "test_max": test_max,
    "test_min": test_min,
    "test_std": test_std,
    "accuracies_train": train_accuracy,
    "accuracies_test": test_accuracy
}

# Writes into the file
os.chdir('../tf-model-testing')
print(os.getcwd())

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

21-10-2024-14-35-42
C:\NN-P4\tf-model-testing


In [10]:
end = time.time()
print(end - start)

118.81576704978943
